In [1]:
import numpy as np
import pandas as pd

from dax.models.combination import combine_models
from energy_consumption.models.combination import combine_selected_models

In [ ]:
energyforecasts = combine_selected_models()
energyforecasts

In [3]:
if 'date_time' in energyforecasts.columns:
    energyforecasts_m2 = energyforecasts.drop(columns=['date_time'])
energyforecasts_m2

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-07,energy,36 hour,67.97968,69.725767,70.301619,70.978373,72.202043
1,2024-02-07,energy,40 hour,63.451999,66.388132,66.990599,67.077972,67.545582
2,2024-02-07,energy,44 hour,58.342858,61.253161,63.085771,65.179048,68.041755
3,2024-02-07,energy,60 hour,55.279575,59.033232,60.314479,62.600259,66.176225
4,2024-02-07,energy,64 hour,51.446333,55.545295,57.09232,58.750127,62.173967
5,2024-02-07,energy,68 hour,51.905394,54.281139,55.296477,57.825593,60.233447


In [4]:
from dax.help_functions.get_dax_data import get_prepared_data, get_data
daxdata = get_data()

In [ ]:
df_sub_dax = combine_models(daxdata)
df_sub_dax

In [11]:
from datetime import datetime
import pandas as pd 

weeks = ['0 week', '1 week', '2 week', '3 week', '4 week']
date_st = (datetime.today().strftime('%Y-%m-%d'))

df_sub_inf = pd.DataFrame({
    "forecast_date": date_st,
    "target": "infections",
    "horizon": weeks,
    "q0.025": np.nan,
    "q0.25": np.nan,
    "q0.5": np.nan,
    "q0.75": np.nan,
    "q0.975": np.nan})

In [12]:
submission_frame = pd.merge(df_sub_dax, energyforecasts_m2, how='outer')
submission_frame = pd.merge(submission_frame, df_sub_inf, how='outer')

In [ ]:
submission_frame = submission_frame.drop(columns={'date_time'})

In [14]:
submission_frame

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-07,DAX,1 day,-2.174163,-0.503785,0.099792,0.652692,1.91608
1,2024-02-07,DAX,2 day,-1.55725,-0.441126,0.051752,0.544629,1.660754
2,2024-02-07,DAX,5 day,-1.877683,-0.521228,0.062166,0.64556,2.002014
3,2024-02-07,DAX,6 day,-3.769115,-0.805395,0.18802,1.152169,3.627718
4,2024-02-07,DAX,7 day,-4.456848,-0.948556,0.31028,1.464725,4.333235
5,2024-02-07,energy,36 hour,67.97968,69.725767,70.301619,70.978373,72.202043
6,2024-02-07,energy,40 hour,63.451999,66.388132,66.990599,67.077972,67.545582
7,2024-02-07,energy,44 hour,58.342858,61.253161,63.085771,65.179048,68.041755
8,2024-02-07,energy,60 hour,55.279575,59.033232,60.314479,62.600259,66.176225
9,2024-02-07,energy,64 hour,51.446333,55.545295,57.09232,58.750127,62.173967


In [46]:
date_string = date_st
date_obj = datetime.strptime(date_string, "%Y-%m-%d")
formatted_date = date_obj.strftime("%Y%m%d")

submission_frame.to_csv(
    str('C:/Users/Maria/Documents/Studium/Pyhton Projekte/PTSFC/submission_files/' +
        formatted_date + '_BrienneVonTarth'),
    index=False, na_rep="NA")